In [1]:
import tensorflow as tf

- 메모리 용량에 맞지 않는 아주 큰 규모의 데이터셋으로 딥러닝 시스템을 훈련해야 하는 경우가 많다.
- 이럴 때 텐서플로우 데이터 API 사용
- 데이터셋 객체를 만들고 데이터를 읽어 올 위치와 변환 방법을 지정해주면 된다.
- 멀티스레딩, 큐, 배치, 프리페치(prefetch)같은 상세한 사항을 모두 대신 처리해 준다.

데이터 API가 읽을 수 있는 포맷
- 텍스트 파일(예를 들면 csv파일)
- 고정 길이의 레코드를 가진 이진 파일
- TFRecord 포맷을 사용하는 이진 파일
    - TFRecord는 길이가 다른 레코드를 지원한다.
    일반적으로 프로토콜 버퍼(protocol buffer)를 담은 유연하고 효율적인 이진 포맷

**프로토콜 버퍼(Protocol Buffers, Protobuf)** 는 구조화된 데이터를 직렬화하는 방식이다. 유선이나 데이터 저장을 목적으로 서로 통신할 프로그램을 개발할 때 유용하다.

**직렬화** 또는 **시리얼라이제이션(serialization)** 은 컴퓨터 과학의 데이터 스토리지 문맥에서 데이터 구조나 오브젝트 상태를 동일하거나 다른 컴퓨터 환경에 저장(이를테면 파일이나 메모리 버퍼에서, 또는 네트워크 연결 링크 간 전송)하고 나중에 재구성할 수 있는 포맷으로 변환하는 과정이다.
오브젝트를 직렬화하는 과정은 오브젝트를 마샬링한다고도 한다. 반대로, 일련의 바이트로부터 데이터 구조를 추출하는 일은 **역직렬**화 또는 **디시리얼라이제이션(deserialization)** 이라고 한다.

또한 SQL DB에서 읽는 기능도 지원한다.\
구글 빅쿼리와 같은 다양한 데이터 소스에서 읽을 수 있는 오픈 소스도 있다.

대용량 데이터를 효율적으로 읽는 것은 단지 어렵기만 한 것이 아니다.\
일반적으로 정규화 같은 데이터 전처리가 필요하다.\
또한 간편한 수치형 필드로만 구성되지 않는다. 텍스트 특성이나 범주형 특성 등도 있다.\
이런 특성들은 원-핫 인코딩, BoW 인코딩, 임베딩(embedding)등을 사용하여 인코딩되어야 한다.\
이런 모든 전처리 과정을 처리하기 위해 **사용자 정의 전처리 층**을 만드는 방법이 있다. 케라스에서 젝홍하는 표준 전처리 층을 사용하는 방법이 있다

- 데이터API, TFRecord 포맷
- 사용자 정의 전처리 층 만드는 방법
- 표준 케라스 전처리 층 사용법

- TF 변환(tf.Transform)
    - (실행 속도를 높이기 위해) 훈련 전에 전체 훈련 세트에 대해 실행하는 전처리 함수를 작성할 수 있다. 그다음 텐서플로 함수로 변환하고 상용 환경에 배포된 다음 훈련된 모델과 협업하여 새로운 샘플에 대해 동적으로 전처리를 수행할 수 있다.
    
- TF 데이터셋 (TFDS)
    - 각종 데이터셋을 다운로드할 수 있는 편리한 함수를 제공한다. 이미지넷과 같은 대용량 데이터셋도 포함된다. 또한 데이터 API로 조작할 수 있는 편리한 데이터셋 객체도 제공한다.

# TF 입력 파이프라인 빌드
https://www.tensorflow.org/guide/data?hl=ko

## 데이터 API (tf.data)

- 전체적인 데이터 API의 중심에는 데이터셋 개념이 있다. 이는 연속된 데이터 샘플을 나타낸다.
- 일반적으로 디스크에서 데이터를 점진적으로 읽는 데이터셋을 사용

간단하고 재사용 가능한 조각으로 복잡한 입력 파이프라인을 빌드할 수 있다.\
예를 들어 이미지 모델의 파이프라인은\
분산된 파일 시스템의 파일에서 데이터를 집계하고 각 이미지에 임의의 퍼터베이션을 적용하며 무작위로 선택한 이미지를 학습을 위한 batch로 병합할 수 있다.

> 텍스트 모델의 파이프라인에는 원시 텍스트 데이터에서 심볼을 추출하고, 이를 룩업 테이블이 있는 embedding 식별자로 변환하며, 길이가 서로 다른 시퀀스를 batch 처리하는 과정이 포함될 수 있다.

> 퍼터베이션 : 극도로 평온한 상태에 있는 물리계에 약간의 불균형을 주는 것을 말한다.

메모리에서 전체 데이터셋 생성

In [4]:
X = tf.range(10)
X

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [7]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
